In [1]:
import subprocess
import os

In [14]:
def get_data(file_name,p = 0.6,add_intent = False):
#     with open("./data/{}.train".format(file_name),'r') as f:
    with open("./data/train_data/{}.train".format(file_name),'r') as f:

        lines = f.readlines()

    train_x = []
    train_y = []
    x = []; y = []
    for line in lines:
        line = line.strip().split('\t')
        if(len(line)>1):
            x.append(line[0])
            y.append(line[1])
        else:
            if add_intent:
                x.append('intent');y.append('B-'+file_name+'_intent')
            train_x.append(x);train_y.append(y)
            x = [];y = []
    if(x!=[]):
        if add_intent:
            x.append('intent');y.append('B-'+file_name+'_intent')
        train_x.append(x);train_y.append(y)
    nums = len(train_x)
    x_train = train_x[0:int(p*nums)];x_valid = train_x[int(0.8*nums):int(0.9*nums)];x_test = train_x[int(0.9*nums):]
    y_train = train_y[0:int(p*nums)];y_valid = train_y[int(0.8*nums):int(0.9*nums)];y_test = train_y[int(0.9*nums):]
#     x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,test_size=0.1,random_state = 0)
#     x_train,x_valid,y_train,y_valid = train_test_split(x_train,y_train,test_size = 0.1,random_state = 0)
    return x_train,x_valid,x_test,y_train,y_valid,y_test

In [15]:
def process_data(x_train,x_valid,x_test,y_train,y_valid,y_test,file_name,p=0.6):
    lines = []
    for i in range(len(x_train)):
        for j in range(len(x_train[i])):
            lines.append(x_train[i][j]+' '+y_train[i][j]+'\n')
        lines.append('\n')
    with open("./data/crf_data/{}_{}.train.crf".format(file_name,str(p)),'w') as f:
        f.write(''.join(lines))
    lines = []    
    for i in range(len(x_test)):
        lines.append('\n'.join(x_test[i])+'\n\n')
    with open("./data/crf_data/{}_{}.test.crf".format(file_name,str(p)),'w') as f:
        f.write(''.join(lines))

In [18]:
def get_combined_data(p=0.6,add_intent = True):
    x_train = [];x_valid=[];x_test=[];y_train=[];y_valid=[];y_test=[]
    for file_name in ['ask_price','time_delay']:
        x_train_t,x_valid_t,x_test_t,y_train_t,y_valid_t,y_test_t = get_data(file_name,p,add_intent)
        x_train.extend(x_train_t);x_valid.extend(x_valid_t);x_test.extend(x_test_t)
        y_train.extend(y_train_t);y_valid.extend(y_valid_t);y_test.extend(y_test_t)
    return x_train,x_valid,x_test,y_train,y_valid,y_test

In [19]:
def get_combined_data_sep_label(p=0.6):
    x_train = [];x_valid=[];x_test=[];y_train=[];y_valid=[];y_test=[]
    for file_name in ['ask_price','time_delay','price']:
        x_train_t,x_valid_t,x_test_t,y_train_t,y_valid_t,y_test_t = get_data(file_name,p)

        y_train_t = [[i.replace('price','price.{}'.format(file_name)) for i in sample] for sample in y_train_t]
        y_valid_t = [[i.replace('price','price.{}'.format(file_name)) for i in sample] for sample in y_valid_t]
        y_test_t = [[i.replace('price','price.{}'.format(file_name)) for i in sample] for sample in y_test_t]
        
        x_train.extend(x_train_t);x_valid.extend(x_valid_t);x_test.extend(x_test_t)
        y_train.extend(y_train_t);y_valid.extend(y_valid_t);y_test.extend(y_test_t)
    return x_train,x_valid,x_test,y_train,y_valid,y_test

In [20]:
for file_name in ['ask_price','time_delay','price']:
    for p in [0.4,0.8]:
        process_data(*get_data(file_name,p),file_name,p)

In [21]:
for p in [0.4,0.8]:
    process_data(*get_combined_data(p),'combine_same',p)
    process_data(*get_combined_data_sep_label(p),'combine_diff',p)

In [225]:
def evaluate_crf(file_name,p=0.6):
    x_train,x_valid,x_test,y_train,y_valid,y_test = get_data(file_name,p)
    with open("./data/crf_data/re/{}_{}.re.crf".format(file_name,str(p)),'r') as f:
        lines = f.readlines()
    test_re = []
    y = []
    for line in lines:
        line = line.strip().split('\t')
        if(len(line)>1):
            y.append(line[1])
        else:
            test_re.append(y)
            y =[]
    if(y!=[]):
        test_re.append(y)
    print(classification_report(y_test,test_re,digits=4))
    if(file_name!=''):
        pred_test = test_re
        test_re = [list(zip(x_test[i],y_test[i],pred_test[i])) for i in range(len(x_test))]
        for i in range(len(test_re)):
            test_re[i] = ['\t'.join(t) for t in test_re[i]]
            test_re[i] = '\n'.join(test_re[i]) + '\n'
        with open('./data/test_re/{}_{}_predict_crf.txt'.format(file_name,p),'w') as f:
            f.write('\n'.join(test_re)) 

In [226]:
def evaluate_crf_combine(x_train,x_valid,x_test,y_train,y_valid,y_test,file_name,p=0.6):
    with open("./data/crf_data/re/{}_{}.re.crf".format(file_name,str(p)),'r') as f:
        lines = f.readlines()
    test_re = []
    y = []
    for line in lines:
        line = line.strip().split('\t')
        if(len(line)>1):
            y.append(line[1])
        else:
            test_re.append(y)
            y =[]
    if(y!=[]):
        test_re.append(y)
    print(classification_report(y_test,test_re,digits=4))

In [250]:
for file_name in ['ask_price','time_delay','price','combine_same','combine_diff']:
    for p in [0.4,0.8]:
        train_data_path = "./data/crf_data/{}_{}.train.crf".format(file_name,str(p))
        test_data_path = "./data/crf_data/{}_{}.test.crf".format(file_name,str(p))
        pred_path =  "./data/crf_data/re/{}_{}.re.crf".format(file_name,str(p))

        model_name = './data/crf_data/model/{}_{}.model'.format(file_name,str(p))
#         crf_train = "crf_learn -f 3 -c 4.0 crf++/template.txt {} {}".format(train_data_path, model_name)
#         process = subprocess.Popen(crf_train.split(), stdout=subprocess.PIPE)
#         output = process.communicate()[0]     
        crf_train = "crf_learn -f 3 -c 4.0 crf++/template.txt {} {}".format(train_data_path, model_name)
        crf_test = "crf_test -m {} {} > {}".format(model_name,test_data_path,pred_path)     
        os.system(crf_train)
        os.system(crf_test)
#         process = subprocess.Popen(crf_test, stdout=subprocess.PIPE)
#         output = process.communicate()[0]

In [251]:
# crf_learn -f 3 -c 4.0 template.txt ../data/crf_data/price_0.6.train.crf price_model
# crf_test -m price_model ../data/crf_data/price_0.6.test.crf > price_0.6_re.txt

In [252]:
for file_name in ['ask_price','time_delay','price']:
    evaluate_crf(file_name,p=0.4)

           precision    recall  f1-score   support

    price     0.9202    0.8099    0.8615       242

micro avg     0.9202    0.8099    0.8615       242
macro avg     0.9202    0.8099    0.8615       242

           precision    recall  f1-score   support

    price     0.9559    0.9378    0.9468       370

micro avg     0.9559    0.9378    0.9468       370
macro avg     0.9559    0.9378    0.9468       370

           precision    recall  f1-score   support

    price     0.8196    0.8275    0.8235       313

micro avg     0.8196    0.8275    0.8235       313
macro avg     0.8196    0.8275    0.8235       313



In [253]:
for file_name in ['ask_price','time_delay','price']:
    evaluate_crf(file_name,p=0.8)

           precision    recall  f1-score   support

    price     0.9286    0.8595    0.8927       242

micro avg     0.9286    0.8595    0.8927       242
macro avg     0.9286    0.8595    0.8927       242

           precision    recall  f1-score   support

    price     0.9591    0.9514    0.9552       370

micro avg     0.9591    0.9514    0.9552       370
macro avg     0.9591    0.9514    0.9552       370

           precision    recall  f1-score   support

    price     0.8449    0.8530    0.8490       313

micro avg     0.8449    0.8530    0.8490       313
macro avg     0.8449    0.8530    0.8490       313



In [234]:
for p in [0.4,0.8]:
    evaluate_crf_combine(*get_combined_data(p),'combine_same',p)
    evaluate_crf_combine(*get_combined_data_sep_label(p),'combine_diff',p)

           precision    recall  f1-score   support

    price     0.8955    0.8897    0.8926       925

micro avg     0.8955    0.8897    0.8926       925
macro avg     0.8955    0.8897    0.8926       925

                  precision    recall  f1-score   support

price.time_delay     0.8469    0.7027    0.7681       370
 price.ask_price     0.7243    0.5537    0.6276       242
     price.price     0.5814    0.7188    0.6429       313

       micro avg     0.7042    0.6692    0.6863       925
       macro avg     0.7250    0.6692    0.6890       925

           precision    recall  f1-score   support

    price     0.8980    0.8854    0.8917       925

micro avg     0.8980    0.8854    0.8917       925
macro avg     0.8980    0.8854    0.8917       925

                  precision    recall  f1-score   support

price.time_delay     0.8509    0.7405    0.7919       370
 price.ask_price     0.7616    0.5413    0.6329       242
     price.price     0.6154    0.7923    0.6927       313

 